In [1]:
import geojson
from glob import glob
import json
import os, sys, fnmatch
from os import makedirs, path as op
import shutil
from PIL import Image
import numpy as np

In [2]:
def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

In [4]:
def get_bounding_box(geoj):
    with open(geoj) as f:
        xcoords = []
        ycoords = []
        data = json.load(f)
        for feature in data['features']:
            geom = feature['geometry']
            for coord in geom['coordinates']:
                if type(coord) == float:  # then its a point feature
                    xcoords.append(geom['coordinates'][0])
                    ycoords.append(geom['coordinates'][1])
                elif type(coord) == list:
                    for c in coord:
                        if type(c) == float:  # then its a linestring feature
                            xcoords.append(coord[0])
                            ycoords.append(coord[1])
                        elif type(c) == list:  # then its a polygon feature
                            xcoords.append(c[0])
                            ycoords.append(c[1])
            coords = np.array(list(geojson.utils.coords(geom)))
    return [min(xcoords), min(ycoords), max(xcoords), max(ycoords)]

In [3]:
geo_dir = ''

In [5]:
root_dir = ''

In [8]:
imagery_dir = ''

In [ ]:
tif_tiles = glob(f'{imagery_dir}/**/*.tif',recursive=True)

for tif in tif_tiles:
    filename_split = os.path.splitext(tif) 
    filename_zero, fileext = filename_split 
    basename = os.path.basename(filename_zero) 
    basename_cp = basename
    basename = basename[15:]
    basename =basename.replace('_3B_AnalyticMS','')
    basename = basename[0:8]+'_'+basename[9:]
    
    try:
        geoj = f'{geo_dir}/{basename}.geojson'             

        bbox = get_bounding_box(geoj)

        config = f'{root_dir}/config.json'

        with open(config, 'r') as con_j:
            config_json = json.load(con_j)

        s = basename[-7:]  
        print(s)             
        st = find(f'*{s}*', imagery_dir)
        print(st)
        config_json['bounding_box'] = bbox
        config_json['geojson'] = geoj
        config_json['imagery'] = f'{imagery_dir}/{basename_cp}.tif'
        print(config_json)  

        #makedirs(f'{root_dir}/configs/')             
        makedirs(f'{root_dir}/configs/{basename}')             
        config_out = f'{root_dir}/configs/{basename}/config.json'

        with open(config_out, 'w') as con_n_j:
            json.dump(config_json, con_n_j)
            print(config_json)

        os.chdir(f'{root_dir}/configs/')           
        cmd = "./batch_lblmkr.sh"           
        os.system(cmd) 
        os.chdir(f'{root_dir}')  
        makedirs(f'data_working_nir/{basename}')
        shutil.move(f'configs/{basename}/data/labels.npz', f'data_all/{basename}')
        shutil.move(f'configs/{basename}/data/tiles', f'data_all/{basename}/tiles')
        tifs = glob(f'data_all/{basename}/tiles'+'/**/*.tif',recursive=True)
        for i in tifs:
            im = Image.open(i)
            im.save(i[:-4]+'.jpg', "JPEG")
        for f in tifs:
            os.remove(f)
                 
    except:
        print(f"no geo for{basename}")